# Fetch co-ordinates of Neighborhood for Clustering neighborhoods in Toronto

Lets import beautiful soup, requests library and pandas

In [34]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

Now read html and store it in a variable

In [35]:
html_source=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup=BeautifulSoup(html_source,'lxml')

In the following code, we would use beautifulsoup library to fetch text inside each tr block, assign each td block into variables named postcode, borough and neighborhood.
A list of dictionaries is used to store initial data
Another dictionary is used, whose key is postcode and value as row_index in the row_list, to complete the requirement that information of many postcode should be present in an single row separated by comma
hence, list of dictionaries can be updated whenever a postcode is repeated because it's index is stored in another dictionary.

Finally creating the intended pandas dataframe we need.

In [36]:
table_text=soup.find('table', class_="wikitable")
row_data=table_text.find_all('tr')
#row_data
#row_data.td.text
rows_list=[]
postNeighborDict={}
row_index=0
for row_data in table_text.find_all('tr'):
    cell_datas=row_data.find_all('td')
    if len(cell_datas)==3:
        postcode=cell_datas[0].text
        borough=cell_datas[1].text
        neighborhood=cell_datas[2].text
        neighborhood=neighborhood.replace('\n','')
        #print(postcode, "==", borough, "--", neighborhood)
        if borough!="Not assigned":
            if neighborhood=="Not assigned":
                neighborhood=borough
            if postcode in postNeighborDict:
                index=postNeighborDict[postcode]
                neighborhood=rows_list[index]['Neighborhood']+','+neighborhood
                rows_list[index]['Neighborhood']=neighborhood
            else:
                dict={'PostalCode': postcode,'Borough': borough,'Neighborhood': neighborhood}
                postNeighborDict[postcode]=row_index
                rows_list.append(dict)
                row_index=row_index+1
postalcode_df=pd.DataFrame(rows_list,columns=['PostalCode','Borough','Neighborhood'])

In [4]:
postalcode_df

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge,Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens,Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson,Garden District"


Let's print the size of this dataframe

In [37]:
print(postalcode_df.shape)

(103, 3)


Let's create a new dataframe postalcode_df2 having two new columns Latitude and Longitude

In [38]:
postalcode_df2=pd.DataFrame(columns=['PostalCode','Borough','Neighborhood','Latitude','Longitude'])
postalcode_df2[['PostalCode','Borough','Neighborhood']]=postalcode_df[['PostalCode','Borough','Neighborhood']]


Now GeoCode api is not working in my case, Hence let's download csv file containing locations.

In [39]:
!wget -q -O 'Geospatial_data.csv' https://cocl.us/Geospatial_data
print('file downloaded')

file downloaded


Read csv file, Store the data in pandas dataframe  

In [40]:
geospatial_df = pd.read_csv('Geospatial_data.csv')
geospatial_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Let's fetch latitude longitude on the basis of postal code

In [41]:
for index_value in range(103):
    latlng_value=geospatial_df.loc[ geospatial_df['Postal Code'] == postalcode_df2.loc[index_value]['PostalCode']]
    latitude=latlng_value.iloc[0]['Latitude']
    longitude=latlng_value.iloc[0]['Longitude']
    postalcode_df2.loc[index_value]['Latitude']=latitude
    postalcode_df2.loc[index_value]['Longitude']=longitude
    #print("index_value : ",index_value, "Latitude: ", latitude, ": Longitude", longitude)

The complete dataframe we obtain is :

In [42]:
postalcode_df2

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7533,-79.3297
1,M4A,North York,Victoria Village,43.7259,-79.3156
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.6543,-79.3606
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.7185,-79.4648
4,M7A,Queen's Park,Queen's Park,43.6623,-79.3895
5,M9A,Etobicoke,Islington Avenue,43.6679,-79.5322
6,M1B,Scarborough,"Rouge,Malvern",43.8067,-79.1944
7,M3B,North York,Don Mills North,43.7459,-79.3522
8,M4B,East York,"Woodbine Gardens,Parkview Hill",43.7064,-79.3099
9,M5B,Downtown Toronto,"Ryerson,Garden District",43.6572,-79.3789
